## Treniranje modela
Ucitavamo sve potrebne biblioteke

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

ucitavamo podatke iz foldera data u kome se nalaze fotografije podijeljene i trening i test skup (naravno ako pokrecete naravno vidite koji je tacno path tih foldera) 

In [3]:


train_dir = 'Data/train'
test_dir = 'Data/test'
#generisemo dataset od fotografija i stavljamo da vrijednost boje u pikselima bude na skali od 0 do 1, a ne od 0 do 255
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_generator = train_datagenerator.flow_from_directory(
        train_dir, #putanja
        target_size=(48,48),#velicina fotografija
        batch_size=64,#broj instanci koje uzimamo prilikom jednog koraka gradijentnog spusta
        color_mode="grayscale",
        class_mode='categorical') 

test_generator = test_datagenerator.flow_from_directory(
        test_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


Pravimo arhitekturu modela, dodajemo sloj po sloj,  o detaljima cemo komentarisati kad zavrsimo uvodi tekst o cnn i fcnn

In [4]:


recognition_model = Sequential()

recognition_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
recognition_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
recognition_model.add(MaxPooling2D(pool_size=(2, 2)))
recognition_model.add(Dropout(0.25))

recognition_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
recognition_model.add(MaxPooling2D(pool_size=(2, 2)))
recognition_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
recognition_model.add(MaxPooling2D(pool_size=(2, 2)))
recognition_model.add(Dropout(0.25))
#fcnn dio arhitekture
recognition_model.add(Flatten())
recognition_model.add(Dense(1024, activation='relu'))
recognition_model.add(Dropout(0.5))
recognition_model.add(Dense(7, activation='softmax'))

In [16]:
recognition_model.summary() 

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 46, 46, 32)        320       
                                                                 
 conv2d_9 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 22, 22, 64)        0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 22, 22, 64)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                 

Treniramo model, ovaj dio programa radi 5 i po sati.

In [ ]:

recognition_model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, decay=1e-6),metrics=['accuracy'])

recognition_model_info = recognition_model.fit_generator(
        train_generator, #trening podaci
        steps_per_epoch=28709 // 64, 
        epochs=60,
        validation_data=test_generator,#test tj validation podaci 
        validation_steps=7178 // 64)

In [ ]:
#cuvamo weights modela u ovom fajlu, i posle to koristimo, i ne moramo vise da pokrecemo gornji kod
recognition_model.save_weights('recogntion_model.h5')
#ja sam ih ubacio u folder tako da vi u sustini ne morate da pokrecete, osim ako necete da nacrtate grafik sledeci

#ovo je crtanje grafika accuracy na trening i test skupu, meni su se izgubili podaci, pa nemam nacrtano sad
plt.title('Model Accuracy')
plt.plot(recognition_model_info.history['accuracy'])
plt.plot(recognition_model_info.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
